#  <center>Tabster - A Tabular Expression Toolbox REST API<center>
### <center><i>Muntazir Fadhel</i><center> <center>McMaster University, Hamilton, Ontario, Canada L8S 4K1<center>

# 1 - Introduction
Although Tabular expressions have been successfully used in the develpment of embeddeded and safety critical systems, there exists a lack of tooling support for such expressions. This lack of tooling support prevents the adoption of such expressions in the wider software industry due to the amount of resources and effort required to develop them properly. To help address this issue we have developed Tabster, a REST API capable of determining key properties of tabular expressions. Tabster is capable of determining the completeness and disjointness of a given tabular expression and can provide counter examples when these properties are not maintained. Furthermore, we have also packaged Tabster as a REST API to allow developers from any development environment to consume the functionality of Tabster's REST API as we will soon demonstrate.

The rest of this paper is organized as follows. In Section 2 we present some preliminary information on tabular expressions, SMT-LIB notations, and REST API's. Section 3 discusses the implementation of the Tabster software system in Java and the test cases used to test the sytem. Section 4 presents the documentation for the Tabster REST API as well as a demonstration on how to consume the API.

# 2 - Preliminaries

## 2.1 Tabular Expressions
When designing software for mission critical systems, developers often need to clearly determine what output a program should give for different classes of inputs. It has been shown that tables provide a clear and convenient way to communicate these conditions [1]. Additionally, such tables allow for easier readability of documentation and facilitate the inspection of a set of conditions for disjointness and completeness. Disjointness and completeness are both critical properties for safety critical systems. Disjointness ensures that the specified function is deterministic and completeness ensures that the entire range of possible input has been considered. Fig. 1 demonstrates the format for tabular expressions and a formal specification of the completeness and disjointness properties. For a detailed discussion of tabular expression semantics, we refer the reader to Jin and Parnas [2].

<img src=index.png>
     <center>**Fig. 1. Tabular Expression Format**</center>

## 2.2 - SMT-LIB v2 Exchange Format

We desired a simple solution for automating the verification of tabular expressions based on various key properties as discussed above. To accomplish this task, we utilized the Z3 SMT solver developed by Microsoft to evaluate the logical statements generated by Tabster to verify a given tabular expression. Satisfiability Modulo Theories (SMT) is an area of automated deduction that studies methods for checking the satisfiability of first-order formulas with respect to some logical theory T of interest. In order to communicate with the Z3 solver, Tabster must convert the logical proofs it generates into a satisfiability problems using the SMT-LIB Exchange format first. To solve this problem, we developed an ANTLR based parser to that is capable of interacting with SMT solver using the SMT-LIB Exchange format.

## 2.3 - REST API

A REST API defines a set of functions which allows developers to perform requests and recieve responses using HTTP protocols such as GET and POST. An API that adheres to the principles of REST does not require the client to know anything about the structure of the API. Rather, the server needs to provide whatever information the client needs to interact with the service. One motivation for the packaging of Tabster as a REST API was due to the accessible nature of REST APIs. Because the consumer of the API and the server are independent of each other, they can be developed in entirely different programming languages and environements. We hope this decision leads to the longetivity and evolution of this project. The example below demonstrates a sample REST API call utilizing GitHub's REST API to get information about a GitHub user.



In [36]:
import requests
import json
# we will find the GitHub user with id 'Zir0-93'
url = 'https://api.github.com/users/Zir0-93'
response = requests.get(url)
print(json.dumps(response.json(), indent=4))

{
    "public_repos": 5, 
    "site_admin": false, 
    "subscriptions_url": "https://api.github.com/users/Zir0-93/subscriptions", 
    "gravatar_id": "", 
    "hireable": true, 
    "id": 3152027, 
    "followers_url": "https://api.github.com/users/Zir0-93/followers", 
    "following_url": "https://api.github.com/users/Zir0-93/following{/other_user}", 
    "blog": null, 
    "followers": 1, 
    "location": null, 
    "type": "User", 
    "email": "muntazir@live.ca", 
    "bio": null, 
    "gists_url": "https://api.github.com/users/Zir0-93/gists{/gist_id}", 
    "company": null, 
    "events_url": "https://api.github.com/users/Zir0-93/events{/privacy}", 
    "html_url": "https://github.com/Zir0-93", 
    "updated_at": "2016-03-23T20:22:27Z", 
    "received_events_url": "https://api.github.com/users/Zir0-93/received_events", 
    "starred_url": "https://api.github.com/users/Zir0-93/starred{/owner}{/repo}", 
    "public_gists": 0, 
    "name": "Muntazir Fadhel", 
    "organizations_url"

# 3 Implementation
## 3.1.1 Generating SMT-LIB Descriptions

The first component of Tabster deals with generating SMT-LIB scripts that check for the satisfiability of a given predicate expression. As we will see later, this functionality is required for being able to determine the completeness and disjointness of tabular expressions. In order to generate such descriptions, we first develop an ANTLR grammar that is capable of correctly parsing predicate expressions. This important parts of this grammar are provided below:

```
/** A grammar to parse predidcate expressions.
 */
grammar PredicateExpression;

compilationUnit
    :   expression EOF
    ;
    
expression 
    : relationalExpression (andsign relationalExpression)*
    | relationalExpression (orsign relationalExpression)*    
    | notsign expression
    ;

notsign
	: BANG
	| TILDE
	;
	
andsign
	: AND
	| LOGICAND
	| BITAND
	;
	
orsign
	: OR
	| LOGICOR
	| BITOR
	;
	   
relationalExpression 
    : addingExpression (EQUAL addingExpression)*
    | addingExpression (NOTEQUAL addingExpression)*
    | addingExpression (GT addingExpression)*
    | addingExpression (GE addingExpression)*
    | addingExpression (LT addingExpression)*
    | addingExpression (LE addingExpression)*
    | quantifierExpression
    ;

quantifierExpression
	: '{' quantifierSymbolVariablePair ':' expression '}'
	;
	
quantifierSymbolVariablePair
	: FORALL variable
	| EXISTS variable
	;
	
addingExpression 
    : multiplyingExpression (ADD multiplyingExpression)*
    | multiplyingExpression (SUB multiplyingExpression)*
    ;
    
multiplyingExpression 
    : signExpression (MUL signExpression)*
    | signExpression (DIV signExpression)*
    | signExpression (MOD signExpression)*
    ;
    
signExpression 
    : ((ADD|SUB) | (BANG|TILDE))? primeExpression
    ;

primeExpression 
    : StringLiteral | IntegerLiteral | BooleanLiteral | FloatingPointLiteral | variable | LPAREN expression /* recursion!!! */ RPAREN
    ;
      
variable
    :  Identifier
    ; 
    
    
```

Next, as the ANTLR parse tree listener enters certain parsing rules, we will incrementally develop the SMT-LIB exhange format representation of the original predicate expression. This is possible due to the structure of the SMT-LIB language and the manner in which we have create the grammar file. The conversion process is done by a parse tree listener generated for the grammar above which incrementally develops the SMT-LIB representation of a given expression as various parsing rules are triggered. To demonstrate the basic principle, consider the conversion process of the expression "x > 2" into its corresponding SMT-LIB representation. The following sequence demonstrates the interaction between the parse tree listener and the currently parsed input.


| Stage | Parsed Symbol | Generated SMT-LIB Description |  
|-------|---------------|-------------------------------|
| 0     |             | Assert (                      | 
| 1     | >             | Assert ( ( >                  | 
| 2     | x             | Assert ( ( > x                | 
| 3     | 2             | Assert ( ( > x 2 ))           | 


Furthermore, the code from the parse tree listener to accomplish this in relation to the example above is reproduced below:

```java

    @Override
    public final void enterExpression(final PredicateExpressionParser.ExpressionContext ctx) {
        	
            // register's the > and signifies the creation of a new SMT-LIB (> <var> <var> ) statement...
            
            smtLibDescription.registerExpressionStart(ctx.notsign().getText());
        
    }

    @Override
    public final void exitExpression(final PredicateExpressionParser.ExpressionContext ctx) {

           // registers the end of an expresion, signifies the creation of a closing bracket for the statement ..
           
            smtLibDescription.registerExpressionEnd();
        
    }
    
    @Override
    public final void enterSignExpression(final PredicateExpressionParser.SignExpressionContext ctx) {
       
         // registers the terms 'x' and '2' in the expression
         
         smtLibDescription.registerExpressionStart(ctx.getText());
    }

```

### 3.1.2 Testing Generated SMT-LIB Descriptions

The following test suite was developed to ensure the conversion of predicate expressions into SMT-LIB descriptions that check for satisfiability of the original expression was done correctly. The basic structure of the test is to first develop a string representing a predicate expression. Next, after calling the extractSMTSolverScript() method of the TabsterService class, we assert that the returned String is the correct SMT-LIB exchange format representation of the original expression.

```java

/**
 * Tests Tabster's ability to parse and generate accurate SMT-LIB Descriptions
 * from Strings representing predicate expressions.
 *
 * @author Muntazir Fadhel
 */
public class TabsterPredicateExpressionParserTest {

    @Test
    public void testParsingExpressionWithMixedVarTypes() throws Exception {
    	final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final String unparsedExpression = "((x > 5) || (x - 3) > 6) && false = y && 3.77 < z";
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        expressionVars.add(new SMTFunction("y", null, FunctionType.BOOL));
        expressionVars.add(new SMTFunction("z", null, FunctionType.REAL));
        final String smtLibDescription = TabsterService
                .extractSMTLibSolverScript(unparsedExpression, expressionVars, true, true);
        Assert.assertTrue(smtLibDescription
                .equals("(set-logic AUFLIRA) (set-option :produce-models true) (declare-fun x () Int) (declare-fun y () Bool) (declare-fun z () Real)  (assert (and (or (> x 5 ) (> (- x 3 ) 6 ) ) (= false y ) (< 3.77 z ) ) ) (check-sat) (get-model) (exit)"));
    }
    
    @Test
    public void testParsingExpressionWithFormalLogicSymbols() throws Exception {
    	final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final String unparsedExpression = "((x > 5) ∨ (x - 3) > 6) ∧ false = y ∧ 3.77 < z";
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        expressionVars.add(new SMTFunction("y", null, FunctionType.BOOL));
        expressionVars.add(new SMTFunction("z", null, FunctionType.REAL));
        final String smtLibDescription = TabsterService
                .extractSMTLibSolverScript(unparsedExpression, expressionVars, true, true);
        Assert.assertTrue(smtLibDescription
                .equals("(set-logic AUFLIRA) (set-option :produce-models true) (declare-fun x () Int) (declare-fun y () Bool) (declare-fun z () Real)  (assert (and (or (> x 5 ) (> (- x 3 ) 6 ) ) (= false y ) (< 3.77 z ) ) ) (check-sat) (get-model) (exit)"));
    }     
    
    @Test
    public void testParsingExpressionWithNotSymbol() throws Exception {
    	final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final String unparsedExpression = "!(x > 5 & !(y < 5)) ∨ !(z)";
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        expressionVars.add(new SMTFunction("y", null, FunctionType.REAL));
        expressionVars.add(new SMTFunction("z", null, FunctionType.BOOL));
        final String smtLibDescription = TabsterService
                .extractSMTLibSolverScript(unparsedExpression, expressionVars, true, true);
        Assert.assertTrue(smtLibDescription
                .equals("(set-logic AUFLIRA) (set-option :produce-models true) (declare-fun x () Int) (declare-fun y () Real) (declare-fun z () Bool)  (assert (or (not (and (> x 5 ) (not (< y 5 ) ) ) ) (not z ) ) ) (check-sat) (get-model) (exit)"));
    } 
    
    @Test
    public void testParsingExpressionWithExistentialQuantifierSymbols() throws Exception {
    	final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final String unparsedExpression = "{∃x:(x > 5)}";
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        final String smtLibDescription = TabsterService
                .extractSMTLibSolverScript(unparsedExpression, expressionVars, true, true);
        Assert.assertTrue(smtLibDescription
                .equals("(set-logic AUFLIRA) (set-option :produce-models true) (declare-fun x () Int)  (assert (exists ((x Int)) (> x 5 ) ) ) (check-sat) (get-model) (exit)"));
    }
    
    
    @Test
    public void testParsingExpressionWithUniversalQuantifierSymbols() throws Exception {
    	final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final String unparsedExpression = "{∀x:(x > 5)}";
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        final String smtLibDescription = TabsterService
                .extractSMTLibSolverScript(unparsedExpression, expressionVars, true, true);
        Assert.assertTrue(smtLibDescription
                .equals("(set-logic AUFLIRA) (set-option :produce-models true) (declare-fun x () Int)  (assert (forall ((x Int)) (> x 5 ) ) ) (check-sat) (get-model) (exit)"));
    }
    
   
   @Test
    public void testParsingExpressionWithNestedQuantifierExpression() throws Exception {
    	final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final String unparsedExpression = "{∃x:{∃y:(x > y)}}";
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        expressionVars.add(new SMTFunction("y", null, FunctionType.INT));
        final String smtLibDescription = TabsterService
                .extractSMTLibSolverScript(unparsedExpression, expressionVars, true, true);
        Assert.assertTrue(smtLibDescription
                .equals("(set-logic AUFLIRA) (set-option :produce-models true) (declare-fun x () Int) (declare-fun y () Int)  (assert (exists ((x Int)) (exists ((y Int)) (> x y ) ) ) ) (check-sat) (get-model) (exit)"));
    }
    
```

## 3.2.1 Reading SMT Solver Output

If a given tabular expressions is not complete or disjoint, the smt solver used will provide the user with counter examples that will demonstrate the lack of these two properties. Yet again however, we need to parse the output from the solver in order to get the counter examples we are looking for. The grammer for the output of an smt solver is produced below.

```
/** 
 * Grammar for parsing SMT Lib v2 Model Output String
 */
grammar SMTModel;

compilationUnit
    :   satResult? ('(model' functionDeclaration* ')')?
    ;
    
satResult
	: 'sat'
	| 'unsat'
	;
	
functionDeclaration
    :   '(define-fun' varName '()' varType  varValue  ')' ;

varValue
    :   StringLiteral | '('? IntegerLiteral ')'? | BooleanLiteral | '('? FloatingPointLiteral ')'? | divisionStatement
    ;

exclamationStatement  
	:  Identifier '!' IntegerLiteral
	;
	
divisionStatement
	: '(' '/' FloatingPointLiteral  FloatingPointLiteral ')'
	;

varType
    :   'Int'
    |   'Bool'
    |   'Real' 
    ;
    
varName
    :   Identifier '!' IntegerLiteral
    |	Identifier
    ;
```

A simple parse tree listener is built to listen on the **functionDeclaration** parsing rule to capture the name, type, and value of the variable involved in the SMT solver output.

### 3.2.2 Testing Variable Parsing of SMT Model Output

The following tests were generated to ensure Tabster is able to retrieve variables and their corresponding values from the output produced by an SMT solver when a model is found. The test suite first generates a sample model output and then calls the extractModelFunctionsValues() method of the TabsterService Class. Next, tests are performed to check whether or not Tabster has found the correct variables and their associated values. The code for this test suite is reproduced below:

```java
/**
 * Tests to ensure correct parsing and evaluation of SMT-LIB model output.
 *
 * @author Muntazir Fadhel
 */
public class TabsterSMTModelParserTest {

	private static ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
	
	private static String varXName = "x";
	private static FunctionType varXType = FunctionType.INT;
	private static String varXValue = "(- 6)";
	
	private static String varZName = "z";
	private static FunctionType varZType = FunctionType.REAL;
	private static String varZValueString = "(/ 477.0 100.0)";
	private static String varZValue = "4.77";
	
	private static String varYNameModelString = "y!0";
	private static String varYName = "y";
	private static FunctionType varYType = FunctionType.BOOL;
	private static String varYValue = "false";
    
	private static String varANameModelString = "a!1";
	private static String varAName = "a";
	private static FunctionType varAType = FunctionType.INT;
	private static String varAStringValue = "(/ 477.0 100.0)";
	private static String varAValue = "4";
	
	@BeforeClass
    public static void testParseGeneralSMTModelExpression() throws Exception {

        expressionVars.add(new SMTFunction(varXName, null, varXType));
        expressionVars.add(new SMTFunction(varYName, null, varYType));
        expressionVars.add(new SMTFunction(varZName, null, varZType));
        expressionVars.add(new SMTFunction(varAName, null, varAType));
        
        // String that represents sample SMT Solver model output
        final String smtModelOutpuString = "sat (model(define-fun " + varXName + " () "
        + varXType.getValue() + " " + varXValue + ")(define-fun " + varYNameModelString + " () " 
        		+ varYType.getValue() + " " + varYValue + " )(define-fun " + varZName 
        		+ " () " + varZType.getValue() + " " + varZValueString + ") (define-fun " + varANameModelString 
        		+ " () " + varAType.getValue() + " " + varAStringValue + "))";
       SMTModel model = TabsterService.extractModelFunctionsValues(unparsedExpression, expressionVars);
       expressionVars = model.getFunctions();
    }
    
    @Test
    public void testParsedIntValueCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varXName)) {
    			if (var.getValue().equals(varXValue)) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
    
    @Test
    public void testParsedIntTypeCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varXName)) {
    			if (var.getType().getValue().equals(varXType.getValue())) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
    
    @Test
    public void testParsedRealValueCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varZName)) {
    			if (var.getValue().equals(varZValue)) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
    
    @Test
    public void testParsedRealValueAsDivisionStatementCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varAName)) {
    			if (var.getValue().equals(varAValue)) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
    
    @Test
    public void testParsedRealTypeCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varZName)) {
    			if (var.getType().getValue().equals(varZType.getValue())) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
    
    @Test
    public void testParsedBoolValueCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varYName)) {
    			if (var.getValue().equals(varYValue)) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
    
    @Test
    public void testParsedBoolTypeCorrectly() {
    	
    	boolean pass = false;
    	
    	for (SMTFunction var : expressionVars) {
    		if (var.getVarName().equals(varYName)) {
    			if (var.getType().getValue().equals(varYType.getValue())) {
    				pass = true;
    			}
    		}
    	}
    	Assert.assertTrue(pass);
    }
}
```

### 3.3.1 Checking Completeness of Tabular Expression

To check the completeness of a tabular expression, Tabster must ensure that the entire range of possible input values is covered for every variable in the table. Formally, this is expressed as (3) in Figure 1. Therefore, in order to test completeness Tabster will create an overall expression consisting of the negated version of each expression in a table all joined together using the logical conjunction symbol. If the resulting expression is satisfiable, the tabular expression is not complete and the solver will return a set of counter examples indicating the lack of completeness in the table. However, if the resulting expression is not satisfiable the given tabular expression is complete. The code to check for completeness using this method is provided below.

```java

/**
     * The given tabular expression is complete when the returned model
     * is not satisfiable, otherwise the expression is incomplete. If the 
     * expression is incomplete, the values of the SMTFunctions within the
     * model will include counter examples.
     *
     * @param tabularExpression
     *           The list of individuals expressions that form the cells of the tabular expression
     * @param expressionVars
     *           The list of variables used in the tabular expression for which to check completeness for
     * @return SMTModel
     * If the model is SAT, the expression is not complete and the returned object will contain counter examples, otherwise  the given tabular expression is complete.
     *
     */
    public static SMTModel determineTabularExpressionCompleteness(ArrayList<String> tabularExpression, ArrayList<SMTFunction> expressionVars) throws Exception {

    	// to check for completion, check if the following expression is satisfiable:
    	String completionCheckExpression = "";
    	for (String predExpr : tabularExpression) {
            
            // continuously append the logical disjunctive between the cells in the table ...
    		completionCheckExpression += "!(" + predExpr + ") ∧ ";
    	}
    	// remove the last LOGICAL AND symbol
    	completionCheckExpression = completionCheckExpression.substring(0, completionCheckExpression.length() - 2);
        
        // extract the SMT LIB script to determine satisfiability of the generated expression
    	String SMTLibString = extractSMTLibSolverScript(completionCheckExpression, expressionVars, true, true);
    	String scriptResult = runSMTScript(SMTLibString, expressionVars);
        
        // If the SMT Model is SAT, the expression is not complete, otherwise the expression is complete.
    	SMTModel model = extractModelFunctionsValues(scriptResult, expressionVars);
    	return model;
    }
```

# 3.3.3 Testing Completeness Check of Tabular Expressions

The following test suite was developed to ensure Tabster correctly determines the completeness and consistency of inputted tabular expressions. The tests first generate a tabular expression and then call the determineTabularExpressionCompleteness() method of the TabsterService class. Next, we assert whether the tabular expression was complete or not by checking the satisfiability of the resulting SMT model. The code for the test suite is provide below.

```java


@Test
	public void testSimpleCompleteTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("x > 0");
		tabularExpression.add("x < 0");
		tabularExpression.add("x = 0");   
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		final boolean isComplete = !TabsterService.determineTabularExpressionCompleteness(tabularExpression, expressionVars).isSat();
		Assert.assertTrue(isComplete == true);
	}
    
    @Test
	public void testSimpleIncompleteTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("x > 5");
		tabularExpression.add("x < 5");   
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		SMTModel model = TabsterService.determineTabularExpressionCompleteness(tabularExpression, expressionVars);
		final boolean isComplete = !model.isSat();
		Assert.assertTrue(isComplete == false);
		Assert.assertTrue(model.getFunctions().get(0).getValue().equals("5"));
	}
    
    
    
	@Test
	public void testComplexCompleteTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("{∃x:(x > 5)}"); 
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		final boolean isComplete = !TabsterService.determineTabularExpressionCompleteness(tabularExpression, expressionVars).isSat();
		Assert.assertTrue(isComplete == true);
	}
    
    @Test
	public void testComplexIncompleteTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("{∀x:(x > 5)}"); 
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		SMTModel model = TabsterService.determineTabularExpressionCompleteness(tabularExpression, expressionVars);
		final boolean isComplete = !model.isSat();
		Assert.assertTrue(isComplete == false);
		Assert.assertTrue(model.getFunctions().get(0).getValue().equals("0"));
	}
    
    @Test
	public void testLargeIncompleteTabularExpression() throws Exception {
		
        final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
        final ArrayList<String> expressions = new ArrayList<String>();
        expressions.add("((x < 0) & (y = 3))");
        expressions.add("((x < 0) & (y > 3))");
        expressions.add("(x = 0) & (y < 3)");
        expressions.add("(x = 0) & (y = 3)");
        expressions.add("(x = 0) & (y > 3)");
        expressions.add("(x > 0) & (y < 3)");
        expressions.add("(x > 0) & (y = 3)");
        expressions.add("(x > 0) & (y > 3)");
        //expressions.add("(x < 0) & (y < 3)");
        expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
        expressionVars.add(new SMTFunction("y", null, FunctionType.INT));
        SMTModel model = TabsterService.determineTabularExpressionCompleteness(expressions, expressionVars);
        Assert.assertTrue(model.isSat());
        model = TabsterService.determineTabularExpressionDisjointness(expressions, expressionVars);
        Assert.assertTrue(!model.isSat());
		
	}
    
```
    
    
### 3.4.1 Checking Disjointness of Tabular Expression

To check the disjointness of a tabular expression, Tabster must ensure that the given tabular expression is essentially deterministic. Formally, this is expressed as (2) in Figure 1. Therefore, in order to test disjointness Tabster will compare every individual expression in the table with every other expression in the table. If the resulting expression from joining the two expressions in any given comparison using the logical conjunction symbol is satisfiable, than the tabular expression overall is not deterministic. The code for this method is provided below:

```java

/**
     * If the returned model is satisfiable, the given expression are
     * not disjoint. In this case the returned SMTFunctions will show counter
     * examples. If the model is not satisfiable, then the given tabular
     * expressions is disjoint.
     */
    public static SMTModel determineTabularExpressionDisjointness(ArrayList<String> tabularExpression, ArrayList<SMTFunction> expressionVars) throws Exception {
    	// The basic strategy to determine disjointness is to ensure that no two condition rows
    	// in the table when AND'ed together are satisifiable.
    	for (int i = tabularExpression.size() - 1; i > 0; i --) {
    		for (int j = i - 1; j >= 0; j --) {
    			String checkExpression = tabularExpression.get(i) + " ∧ " + tabularExpression.get(j);
    			// if the problem is satisfiable, that is, the returned input variables have values, the tabular
    			// expression is not deterministic and not disjoint.
    			String SMTLibString = extractSMTLibSolverScript(checkExpression, expressionVars, true, true);
    			String scriptResult = runSMTScript(SMTLibString, expressionVars);
    			SMTModel model = extractModelFunctionsValues(scriptResult, expressionVars);
    			if (model.isSat()) {
                    // table not disjoint!!! return the model indicating counter examples
    				return model;
    			}
    		}
    	}
    	// if we have reached here, expressions are disjoint!
    	SMTModel disjointModel = new SMTModel();
    	disjointModel.setFunctions(expressionVars);
    	disjointModel.setSat(false);
    	return disjointModel;
    }
    
```
    
### 3.4.2 Testing Disjointness Check of Tabular Expressions
 
The tests for determining the disjointness of a given tabular expression are identical in procedure to section 3.3.2, the code for this test suite is provided below.

```java

@Test
	public void testSimpleDisjointTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("x > 5");
		tabularExpression.add("x < 5");
		tabularExpression.add("x = 5");   
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		final boolean isDisjoint = !TabsterService.determineTabularExpressionDisjointness(tabularExpression, expressionVars).isSat();
		Assert.assertTrue(isDisjoint == true);
	}
	
    
    @Test
	public void testSimpleNonDisjointTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("x > 5");
		tabularExpression.add("x < 5");
		tabularExpression.add("x < 8");   
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		SMTModel model = TabsterService.determineTabularExpressionDisjointness(tabularExpression, expressionVars);
		final boolean isDisjoint = !model.isSat();
		Assert.assertTrue(isDisjoint == false);
		Assert.assertTrue(model.getFunctions().get(0).getValue().equals("0"));
	}
    
    
    @Test
	public void testComplexNonDisjointTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("((x > 5) && (y = false))");
		tabularExpression.add("((x < 5) || (y = true))");
		tabularExpression.add("(x = 5) & ((y = true) | (y = false)) ");   
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		expressionVars.add(new SMTFunction("y", null, FunctionType.BOOL));
		SMTModel model = TabsterService.determineTabularExpressionDisjointness(tabularExpression, expressionVars);
		final boolean isDisjoint = !model.isSat();
		Assert.assertTrue(isDisjoint == false);
		Assert.assertTrue(model.getFunctions().get(0).getValue().equals("5"));
		Assert.assertTrue(model.getFunctions().get(1).getValue().equals("true"));
	}
    
    	@Test
	public void testComplexDisjointTabularExpression() throws Exception {
		final ArrayList<SMTFunction> expressionVars = new ArrayList<SMTFunction>();
		final ArrayList<String> tabularExpression = new ArrayList<String>();
		tabularExpression.add("((x > 5) && (y = false))");
		tabularExpression.add("((x <= 5) || (y = true))");  
		expressionVars.add(new SMTFunction("x", null, FunctionType.INT));
		expressionVars.add(new SMTFunction("y", null, FunctionType.BOOL));
		SMTModel model = TabsterService.determineTabularExpressionDisjointness(tabularExpression, expressionVars);
		final boolean isDisjoint = !model.isSat();
		Assert.assertTrue(isDisjoint == true);
	}
    
```
## 4 Tabster REST API

### 4.1 Overview

Call Tabster's REST API with JSON representations of tables to determine the completeness and disjoitness of these tabular expressions. The following section will describe the endpoints that makes up the official Tabster REST API. There is no authorization required and all communication is done using JSON.

### 4.2 HTTP Verbs

Tabster tried to adhere as closely as possible to standard HTTP and REST conventions in its use of HTTP status codes. Current HTTP status codes that are returned include:

| Verb |                                    Usage                                   |
|------|:--------------------------------------------------------------------------:|
| POST | Used to determine the comleteness or disjointness of  a tabular expression |

### 4.3 HTTP Status Codes

Tabster tries to adhere as closely as possible to standard HTTP and REST conventions in its use of HTTP status codes. Current HTTPS status codes that are returned include:

| Status Code     |                                                 Usage                                                 |
|-----------------|:-----------------------------------------------------------------------------------------------------:|
| 200 OK          | The request completed successfully                                                                    |
| 400 Bad Request | The request was malformed. The response body will include an error message providing more information |
| 404 Not Found   | The requested resource does not exist                                                                 |

### 4.4 Errors

Whenever an error response (status code >= 400) is returned, the body will contain a JSON object that describe the problem. The error object has the following structure:

| Path      |  Type  | Description                                  |
|-----------|:------:|----------------------------------------------|
| timestamp | String | The timestamp of the event                   |
| status    | String | The error code                               |
| error     | String | The error message                            |
| exception | String | The exception that was caught                |
| message   | String | A message to help resolve the issue          |
| path      | String | The path invoked which resulted in the error |


### 4.5 Interacting With The API

Tabster will process a **POST** request containing a tabular expression expressed as JSON and return whether the given tabular expression is complete and disjoint. Specifics on building the request are provided in the following sections.

#### 4.6 Index

The index provides the entry point into the service.

``` 
http://134.168.26.141:8080 
```

#### 4.7 Request Payload

Note the payload is the JSON representation of a tabular expression. The first section of the JSON object contains a list of the input variables used in the tabular expression. The second section contains a list of the actual expressions used in the table. A sample request payload consisting of a json representation of a tabular expression is provided below:

``` json

{
   "inputVars":[
      {
         "varName":"x",
         "value":"",
         "type":"INT"
      },
      {
         "varName":"y",
         "value":"",
         "type":"INT"
      }
   ],
   "predicateExpressions":[
      "x > 1 ∧ y < 0",
      "x <= 1 ∧ y < 0",
      "x > 1 ∧ y = 0",
      "x <= 1 ∧ y = 0",
      "x > 1 ∧ y > 0",
      "x <= 1 ∧ y > 0"
   ]
}
```

#### 4.8 Endpoints

The following lists the endpoints within the Tabster api and thier associated functionality

| Endpoint        |                       Functionality                      |
|-----------------|:--------------------------------------------------------:|
| api/v1/tabster/tabular/verify | Checks the given input for completeness and disjointness |


#### 4.9 Tabster REST API Demonstration

##### 4.9.1 Complete And Disjoint Tabular Expression

Now we will consume the Tabster REST API from our python environment to demonstrate its functionality. Consider the following set of predicate expressions: 

$f(x) =
\left\{
    \begin{array}{ll}
        x + y  & \mbox{if } x > 1 \wedge y < 0 \\
        x - y & \mbox{if } x \le 1 \wedge y < 0 \\
        x & \mbox{if } x > 1 \wedge y = 0 \\
        xy & \mbox{if } x \le 1 \wedge y > 0 \\
        y & \mbox{if } x >1 \wedge y > 0 \\
        x/y & \mbox{if } x \le 1 \wedge y > 0 \\
    \end{array}
\right.$

This is equivalent to the following tabular expression:

In [37]:
class SampleTabularExpression():
    def _repr_html_(self):
        html = ["<table width=90%>"]
        html.append("<tr>")
        html.append("<td colspan=2>Condition</td>")
        html.append("<td>Result</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td rowspan=3>x > 1</td>")
        html.append("<td>y < 0</td>")
        html.append("<td>x + y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y > 0</td>")
        html.append("<td>y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y = 0</td>")
        html.append("<td>x</td>")
        html.append("</tr>")
        html.append("<td rowspan=3>x <= 1</td>")
        html.append("<td>y < 0</td>")
        html.append("<td>x - y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y > 0</td>")
        html.append("<td>xy</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y = 0</td>")
        html.append("<td>x / y</td>")
        html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

SampleTabularExpression()

<__main__.SampleTabularExpression instance at 0x00000000041D1488>

Notice this table is both complete and disjoint. The JSON representation of this tabular expression would look as follows:

In [38]:
import json
tabularExpression = {"inputVars":[{"varName":"x","value":"","type":"INT"},{"varName":"y","value":"","type":"INT"}],"predicateExpressions":["x > 1 & y < 0","x <= 1 & y < 0","x > 1 & y = 0","x <= 1 & y = 0","x > 1 & y > 0","x <= 1 & y > 0"]};
# the represntative JSON of the tabular expresion....
print(json.dumps(tabularExpression, indent=4))

{
    "predicateExpressions": [
        "x > 1 & y < 0", 
        "x <= 1 & y < 0", 
        "x > 1 & y = 0", 
        "x <= 1 & y = 0", 
        "x > 1 & y > 0", 
        "x <= 1 & y > 0"
    ], 
    "inputVars": [
        {
            "varName": "x", 
            "type": "INT", 
            "value": ""
        }, 
        {
            "varName": "y", 
            "type": "INT", 
            "value": ""
        }
    ]
}


Now we will now call Tabster's REST API using this JSON representation of the tabular  expression.

In [39]:
import requests
import json

# Now we will call Tabster's REST API to determine the completeness and disjointness of the given tabular expression
url = 'http://134.168.26.141:8080/api/v1/tabster/tabular/verify'
# set the post request content to be the JSON representation of the tabular expression we created earlier
data = {"inputVars":[{"varName":"x","value":"","type":"INT"},{"varName":"y","value":"","type":"INT"}],"predicateExpressions":["x > 1 & y < 0","x <= 1 & y < 0","x > 1 & y = 0","x <= 1 & y = 0","x > 1 & y > 0","x <= 1 & y > 0"]};

# set the headers of the request to accept and send JSON
headers = {'content-type': 'application/json', 'accept': 'application/json'}
# retrieve the response
response = requests.post(url, data=json.dumps(data), headers = headers)
# display the result
print(json.dumps(response.json(), indent=4))

{
    "isDisjoint": true, 
    "counterExamples": null, 
    "isComplete": true
}


As expected, the returned JSON object indicates the given tabular expression is both disjoint and complete.

##### 4.9.1 Incomplete Tabular Expression

Let us now turn our attention to a tabular expression that is not complete. Such an expression could be expressed in the following way:

In [51]:
class SampleTabularExpression():
    def _repr_html_(self):
        html = ["<table width=90%>"]
        html.append("<tr>")
        html.append("<td colspan=2>Condition</td>")
        html.append("<td>Result</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td rowspan=3>x > 1</td>")
        html.append("<td>y < 0</td>")
        html.append("<td>x + y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y > 0</td>")
        html.append("<td>y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y = 0</td>")
        html.append("<td>x</td>")
        html.append("</tr>")
        html.append("<td rowspan=2>x <= 1</td>")
        html.append("<td>y < 0</td>")
        html.append("<td>x - y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y > 0</td>")
        html.append("<td>xy</td>")
        html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

SampleTabularExpression()

<__main__.SampleTabularExpression instance at 0x00000000041DB0C8>

Notice  the lack of the condition covering the case when "y = 0" causes this tabular expression to lack completeness. The JSON representation of the tabular expression above would look like the following:

In [47]:
import json
tabularExpression = {"inputVars":[{"varName":"x","value":"","type":"INT"},{"varName":"y","value":"","type":"INT"}],"predicateExpressions":["x > 1 & y < 0","x <= 1 & y < 0","x > 1 & y = 0","x > 1 & y > 0","x <= 1 & y > 0"]};
# the represntative JSON of the tabular expresion....
print(json.dumps(tabularExpression, indent=4))

{
    "predicateExpressions": [
        "x > 1 & y < 0", 
        "x <= 1 & y < 0", 
        "x > 1 & y = 0", 
        "x > 1 & y > 0", 
        "x <= 1 & y > 0"
    ], 
    "inputVars": [
        {
            "varName": "x", 
            "type": "INT", 
            "value": ""
        }, 
        {
            "varName": "y", 
            "type": "INT", 
            "value": ""
        }
    ]
}


We will now call Tabster's REST API using this JSON representation of the tabular  expression.

In [42]:
import requests
import json

# Now we will call Tabster's REST API to determine the completeness and disjointness of the given tabular expression
url = 'http://134.168.26.141:8080/api/v1/tabster/tabular/verify'
# set the post request content to be the JSON representation of the tabular expression we created earlier
data = {"inputVars":[{"varName":"x","value":"","type":"INT"},{"varName":"y","value":"","type":"INT"}],"predicateExpressions":["x > 1 & y < 0","x <= 1 & y < 0","x > 1 & y = 0","x > 1 & y > 0","x <= 1 & y > 0"]};

# set the headers of the request to accept and send JSON
headers = {'content-type': 'application/json', 'accept': 'application/json'}
# retrieve the response
response = requests.post(url, data=json.dumps(data), headers = headers)
# display the result
print(json.dumps(response.json(), indent=4))

{
    "isDisjoint": true, 
    "counterExamples": [
        {
            "varName": "x", 
            "type": "INT", 
            "value": "1"
        }, 
        {
            "varName": "y", 
            "type": "INT", 
            "value": "0"
        }
    ], 
    "isComplete": false
}


As expected, Tabster indicates the tabular expression is not complete, and provides counter examples indicating the lack of completeness in the table. In this scenario, the tabular expression is not complete when x = 1 and y = 0.

##### 4.9.1 Non-Disjoint Tabular Expression

Let us now turn our attention to a tabular expression that is not disjoint. Such an expression could be expressed in the following way:

In [48]:
class SampleTabularExpression():
    def _repr_html_(self):
        html = ["<table width=90%>"]
        html.append("<tr>")
        html.append("<td colspan=2>Condition</td>")
        html.append("<td>Result</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td rowspan=3>x > 1</td>")
        html.append("<td>y < 0</td>")
        html.append("<td>x + y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y > 0</td>")
        html.append("<td>y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y = 0</td>")
        html.append("<td>x</td>")
        html.append("</tr>")
        html.append("<td rowspan=3>x <= 1</td>")
        html.append("<td>y < 0</td>")
        html.append("<td>x - y</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td >y >= 0</td>")
        html.append("<td>xy</td>")
        html.append("</tr>")
        html.append("<tr>")
        html.append("<td ><b>y > 2</b></td>")
        html.append("<td><b> x / y </b></td>")
        html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

SampleTabularExpression()

<__main__.SampleTabularExpression instance at 0x00000000042F4D48>

Notice the expression "y > 2" causes this tabular expression not to be deterministic and there not disjoint. The JSON representation of the tabular expression above would look like the following:

In [49]:
import json
tabularExpression = {"inputVars":[{"varName":"x","value":"","type":"INT"},{"varName":"y","value":"","type":"INT"}],"predicateExpressions":["x > 1 & y < 0","x <= 1 & y < 0","x > 1 & y = 0","x > 1 & y > 0","x <= 1 & y >= 0","x <= 1 & y > 2"]};
# the represntative JSON of the tabular expresion....
print(json.dumps(tabularExpression, indent=4))

{
    "predicateExpressions": [
        "x > 1 & y < 0", 
        "x <= 1 & y < 0", 
        "x > 1 & y = 0", 
        "x > 1 & y > 0", 
        "x <= 1 & y >= 0", 
        "x <= 1 & y > 2"
    ], 
    "inputVars": [
        {
            "varName": "x", 
            "type": "INT", 
            "value": ""
        }, 
        {
            "varName": "y", 
            "type": "INT", 
            "value": ""
        }
    ]
}


We will now call Tabster's REST API using this JSON representation of the tabular  expression.

In [50]:
import requests
import json

# Now we will call Tabster's REST API to determine the completeness and disjointness of the given tabular expression
url = 'http://134.168.26.141:8080/api/v1/tabster/tabular/verify'
# set the post request content to be the JSON representation of the tabular expression we created earlier
data = {"inputVars":[{"varName":"x","value":"","type":"INT"},{"varName":"y","value":"","type":"INT"}],"predicateExpressions":["x > 1 & y < 0","x <= 1 & y < 0","x > 1 & y = 0","x > 1 & y > 0","x <= 1 & y >= 0","x <= 1 & y > 2"]};

# set the headers of the request to accept and send JSON
headers = {'content-type': 'application/json', 'accept': 'application/json'}
# retrieve the response
response = requests.post(url, data=json.dumps(data), headers = headers)
# display the result
print(json.dumps(response.json(), indent=4))

{
    "isDisjoint": false, 
    "counterExamples": [
        {
            "varName": "x", 
            "type": "INT", 
            "value": "1"
        }, 
        {
            "varName": "y", 
            "type": "INT", 
            "value": "3"
        }
    ], 
    "isComplete": true
}


As expected, Tabster indicates the given tabular expression is not disjoint by providing counter examples of z = 1 and y = 3. For these values the given  table is not deterministic.

### References

1. Wassyng, A., Lawford, M.: Lessons learned from a successful implementation of formal methods in an industrial project. In: Araki, K., Gnesi, S., Mandrioli, D. (eds.) FME 2003. LNCS, vol. 2805, pp. 133–153. Springer, Heidelberg (2003) 7. Jin, Y., Parnas, D.L.: Defining the meaning of tabular mathematical expressions.

2. Jin, Y., Parnas, D.L.: Defining the meaning of tabular mathematical expressions. Science of Computer Programming 75(11), 980–1000 (2010)